In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e7/sample_submission.csv
/kaggle/input/playground-series-s5e7/train.csv
/kaggle/input/playground-series-s5e7/test.csv


In [17]:
train = pd.read_csv('/kaggle/input/playground-series-s5e7/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s5e7/test.csv')

In [18]:
train

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,0,0.0,No,6.0,4.0,No,15.0,5.0,Extrovert
1,1,1.0,No,7.0,3.0,No,10.0,8.0,Extrovert
2,2,6.0,Yes,1.0,0.0,NaN,3.0,0.0,Introvert
3,3,3.0,No,7.0,3.0,No,11.0,5.0,Extrovert
4,4,1.0,No,4.0,4.0,No,13.0,NaN,Extrovert
...,...,...,...,...,...,...,...,...,...
18519,18519,3.0,No,7.0,3.0,No,9.0,7.0,Extrovert
18520,18520,1.0,NaN,6.0,7.0,No,6.0,5.0,Extrovert
18521,18521,7.0,Yes,1.0,1.0,Yes,1.0,NaN,Introvert
18522,18522,NaN,Yes,1.0,0.0,Yes,5.0,2.0,Introvert


In [19]:
binary_columns_train = ['Stage_fear', 'Drained_after_socializing', 'Personality']
numerical_columns_train = ['Time_spent_Alone', 'Social_event_attendance', 'Going_outside', 'Friends_circle_size', 'Post_frequency']
binary_columns_test = ['Stage_fear', 'Drained_after_socializing']
numerical_columns_test = ['Time_spent_Alone', 'Social_event_attendance', 'Going_outside', 'Friends_circle_size', 'Post_frequency']

In [20]:
def preprocess_binary(text):
    if text == 'No':
        return 0
    elif text == 'Yes':
        return 1
    elif text == 'Extrovert':
        return 1
    elif text == 'Introvert':
        return 0
        

def preprocess_df(df, binary_cols, numerical_cols):
    df.bfill(inplace = True)
    for i in binary_cols:
        df[i] = df[i].apply(preprocess_binary)
    #preprocessing numerical columns
    #nothing to preprocess idk why i did all this
    return df

def interaction_score(alonetime, socialattendance):
    socialattendance = socialattendance.replace(0, 0.1)  # or np.nan if you prefer
    return alonetime / socialattendance

def feature_eng(df):
    df['Interaction_score'] = interaction_score(df['Time_spent_Alone'], df['Social_event_attendance'])
    return df


train = preprocess_df(train, binary_columns_train, numerical_columns_train)
train = feature_eng(train)

In [21]:
range(0, len(train))

range(0, 18524)

In [22]:
import xgboost as xgb

model = xgb.XGBClassifier(
    n_estimators=300,         # try 300–1000+
    max_depth=3,              # lower depth reduces overfitting
    learning_rate=0.01,       # slower learning = better generalization
    subsample=0.8,            # use part of data per tree
    colsample_bytree=0.8,     # use part of features per tree
    reg_alpha=0.5,            # L1 regularization
    reg_lambda=1,             # L2 regularization
    random_state=42,
    eval_metric='logloss',
    use_label_encoder=False,
    enable_categorical = True
)

In [23]:
x_train = train.drop(columns = ['id', 'Personality'])
y_train = train['Personality']

In [24]:
model.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=300,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [25]:
train.isnull().sum()

id                           0
Time_spent_Alone             0
Stage_fear                   0
Social_event_attendance      0
Going_outside                0
Drained_after_socializing    0
Friends_circle_size          0
Post_frequency               0
Personality                  0
Interaction_score            0
dtype: int64

In [26]:
test = preprocess_df(test, binary_columns_test, numerical_columns_test)
test = feature_eng(test)

In [27]:
x_test = test.drop(columns = ['id'])
preds = model.predict(x_test)
test['preds'] = preds

In [28]:
test

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Interaction_score,preds
0,18524,3.0,0,7.0,4.0,0,6.0,1.0,0.428571,1
1,18525,3.0,1,0.0,0.0,1,5.0,1.0,30.000000,0
2,18526,3.0,0,5.0,6.0,0,15.0,9.0,0.600000,1
3,18527,3.0,0,4.0,4.0,0,5.0,6.0,0.750000,1
4,18528,9.0,1,1.0,2.0,1,1.0,1.0,9.000000,0
...,...,...,...,...,...,...,...,...,...,...
6170,24694,3.0,0,5.0,5.0,0,9.0,6.0,0.600000,1
6171,24695,8.0,1,2.0,1.0,1,0.0,0.0,4.000000,0
6172,24696,2.0,0,4.0,3.0,0,9.0,7.0,0.500000,1
6173,24697,3.0,0,4.0,4.0,0,11.0,9.0,0.750000,1


In [29]:
personality = {
    0 : 'Introvert',
    1 : 'Extrovert'
}

test['preds'] = test['preds'].map(personality)

In [30]:
test

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Interaction_score,preds
0,18524,3.0,0,7.0,4.0,0,6.0,1.0,0.428571,Extrovert
1,18525,3.0,1,0.0,0.0,1,5.0,1.0,30.000000,Introvert
2,18526,3.0,0,5.0,6.0,0,15.0,9.0,0.600000,Extrovert
3,18527,3.0,0,4.0,4.0,0,5.0,6.0,0.750000,Extrovert
4,18528,9.0,1,1.0,2.0,1,1.0,1.0,9.000000,Introvert
...,...,...,...,...,...,...,...,...,...,...
6170,24694,3.0,0,5.0,5.0,0,9.0,6.0,0.600000,Extrovert
6171,24695,8.0,1,2.0,1.0,1,0.0,0.0,4.000000,Introvert
6172,24696,2.0,0,4.0,3.0,0,9.0,7.0,0.500000,Extrovert
6173,24697,3.0,0,4.0,4.0,0,11.0,9.0,0.750000,Extrovert


In [31]:
submission = pd.DataFrame({
    'id' : test['id'],
    'Personality' : test['preds']
})

In [32]:
submission.to_csv('submission.csv', index = False)